# 05. LoRAによるファインチューニング (学習と永続化)

このノートブックでは、QLoRAによる学習を行い、その成果（アダプタ）を **Google Driveに保存** します。これにより、別の日に行われる統合演習（07）で、学習したモデルを再利用することができます。

## 演習の目的
1. 特定の出力形式（JSON）を学習させる。
2. 学習した重みをGoogle Driveに保存（永続化）する。

## 事前準備
Google Colabのメニュー「ランタイム」→「ランタイムのタイプを変更」で **T4 GPU** を選択してください。

In [ ]:
# 編集禁止セル
import os
import sys
import torch
from google.colab import drive

# 1. Google Driveのマウント (永続保存に必須)
if not os.path.isdir('/content/drive'): drive.mount('/content/drive')

repo_path = '/content/llm_lab'
if os.path.exists(repo_path): !rm -rf {repo_path}
!git clone -b stable-base https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft trl datasets gradio
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

from src.common import load_llm, generate_text
from src.lora import create_lora_model, train_lora
from peft import PeftModel
print('セットアップが完了しました。')

## 1. モデルの準備と学習前の確認

In [ ]:
# 編集禁止セル
base_model, tokenizer = load_llm(use_4bit=True)

question = "『星屑のメモリー』の主人公について教えて。"
instruction = "回答は必ず以下のJSON形式で出力してください。\n{ \"answer\": \"...\", \"confidence\": \"high|medium|low\" }"
prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n{instruction}\n\n### 応答:\n"

print("--- 学習前の回答 (JSON形式が守られないことが多い) ---")
res = generate_text(base_model, tokenizer, prompt, max_new_tokens=128)
print(res.split("### 応答:")[-1].strip())

## 2. LoRA学習の実行とGoogle Driveへの保存

学習したアダプタをGoogle Driveの `llm_lab_outputs/demo_adapter` フォルダに保存します。

In [ ]:
# 編集禁止セル
# 保存先パスの設定 (Google Drive)
adapter_dir = "/content/drive/MyDrive/llm_lab_outputs/demo_adapter"
os.makedirs(adapter_dir, exist_ok=True)

print("1. LoRAアダプタをモデルに追加中...")
lora_model = create_lora_model(base_model)

print("2. 簡易学習を開始します（約3〜5分）...")
train_lora(
    model=lora_model,
    tokenizer=tokenizer,
    train_dataset_path='data/lora/lora_train_sample.jsonl',
    output_dir=adapter_dir,
    max_steps=40,
    learning_rate=5e-5
)

print(f"学習が完了し、Google Driveに保存されました: {adapter_dir}")

## 3. 学習後の効果確認

In [ ]:
# 編集禁止セル
print("--- 学習後の回答 (JSON形式が安定する) ---")
res = generate_text(lora_model, tokenizer, prompt, max_new_tokens=128)
print(res.split("### 応答:")[-1].strip())

## まとめ
Google Driveに保存されたこのアダプタは、次の **07. 統合デモ** で読み込んで使用します。セッションを終了しても大丈夫です。